In [1]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 62.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output

In [3]:
df = pd.read_csv('/content/ventas_moda_españa_2025(1).csv', sep=';')

In [4]:
df['precio_unitario'] = df['precio_unitario'].astype(str).str.replace(',', '.').astype(float)
df['ventas_euros'] = df['ventas_euros'].astype(str).str.replace(',', '.').astype(float)


In [5]:
df.describe()

,unidades_vendidas,precio_unitario,ventas_euros
count,500.00000,500.000000,500.000000
mean,426.12200,60.880420,25875.226240
std,220.63132,46.488951,26036.302354
min,52.00000,9.740000,1262.600000
25%,225.50000,33.257500,10382.615000
50%,410.00000,49.075000,19783.845000
75%,613.50000,66.182500,32477.412500
max,799.00000,264.850000,205258.750000


In [6]:
app = dash.Dash()

app.layout = html.Div([
    html.H1("Dashboard de Ventas de Moda",
            style={
                'text-align': 'center',
                'margin-bottom': '30px',
                'background-color': 'white',
                'padding': '10px',
                'border-radius': '5px'
            }),

    # Filtro de región
    html.Div([
        html.Label("Selecciona Región:", style={'font-weight': 'bold', 'margin-bottom': '10px'}),
        dcc.Dropdown(
            id='region-dropdown-ventas',
            options=[{'label': 'Todas las Regiones', 'value': 'Todas'}] +
                    [{'label': region, 'value': region} for region in sorted(df['region'].unique())],
            value='Todas',
            style={'margin-bottom': '30px'}
        )
    ], style={'width': '300px', 'margin': '0 auto'}),

    # Gráfico
    dcc.Graph(id='ventas-timeline-dinamico')

], style={'background-color': 'white', 'padding': '20px'})


# Callback para actualizar el gráfico
@app.callback(
    Output('ventas-timeline-dinamico', 'figure'),
    Input('region-dropdown-ventas', 'value')
)
def update_ventas_timeline(selected_region):
    # Filtrar datos según región seleccionada
    if selected_region == 'Todas':
        filtered_df = df
        title_suffix = " - Todas las Regiones"
    else:
        filtered_df = df[df['region'] == selected_region]
        title_suffix = f" - {selected_region}"


    ventas_mensual_canal = filtered_df.groupby(['fecha', 'canal'])['ventas_euros'].sum().reset_index()

    # Crear el gráfico de líneas por canal
    fig = px.line(ventas_mensual_canal,
                  x='fecha',
                  y='ventas_euros',
                  color='canal',
                  title=f'Evolución de Ventas por Mes y Canal{title_suffix}',
                  labels={'ventas_euros': 'Ventas (€)', 'fecha': 'Mes', 'canal': 'Canal'},
                  markers=True)

    fig.update_traces(line=dict(width=3))
    fig.update_layout(
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        hovermode='x unified',
        height=800,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig


app.run(debug=True, port=8051,height=700, jupyter_height=1000, jupyter_width=1500)

<IPython.core.display.Javascript object>

In [7]:
app_1 = dash.Dash()

app_1.layout = html.Div([
    html.H1("Análisis de Tallas por Producto",
            style={'text-align': 'center', 'margin-bottom': '30px'}),

    # Filtro de región
    html.Div([
        html.Label("Selecciona Región:", style={'font-weight': 'bold', 'margin-bottom': '20px'}),
        dcc.Dropdown(
            id='region-dropdown',
            options=[{'label': 'Todas las Regiones', 'value': 'Todas'}] +
                    [{'label': region, 'value': region} for region in sorted(df['region'].unique())],
            value='Todas',
            style={'margin-bottom': '30px'}
        )
    ], style={'width': '300px', 'margin': '0 auto'}),

    # Gráfico
    dcc.Graph(id='heatmap-tallas'),]
    , style={'background-color': 'white', 'padding': '20px'})

# Callback para actualizar el gráfico
@app_1.callback(
    Output('heatmap-tallas', 'figure'),
    Input('region-dropdown', 'value')
)
def update_heatmap(selected_region):
    # Filtrar datos según región seleccionada
    if selected_region == 'Todas':
        filtered_df = df
        title_suffix = " - Todas las Regiones"
    else:
        filtered_df = df[df['region'] == selected_region]
        title_suffix = f" - {selected_region}"

    # Crear tabla cruzada de talla vs producto
    heatmap_data = filtered_df.groupby(['producto', 'talla'])['unidades_vendidas'].sum().reset_index()

    # Crear pivot table para el heatmap
    pivot_table = heatmap_data.pivot(index='producto', columns='talla', values='unidades_vendidas')
    pivot_table = pivot_table.fillna(0)

    # Reordenar las columnas de talla en orden lógico
    talla_order = ['XS', 'S', 'M', 'L', 'XL', 'XXL']
    pivot_table = pivot_table.reindex(columns=talla_order)

    # Crear el heatmap
    fig = px.imshow(pivot_table,
                    title=f'Unidades Vendidas: Producto vs Talla{title_suffix}',
                    labels=dict(x="Talla", y="Producto", color="Unidades"),
                    aspect="auto",
                    color_continuous_scale='Blues')

    # Personalizar el heatmap
    fig.update_layout(
        title_font_size=18,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        height=600,
        width=900)

    return fig

app_1.run(debug=True, port=8052,jupyter_height=1000, jupyter_width=1500)

<IPython.core.display.Javascript object>

In [8]:
# Preparar datos para Sankey
def prepare_sankey_data(df):
    sankey_df = df.groupby(['campaña', 'canal', 'region'])['ventas_euros'].sum().reset_index()

    # Crear listas para cada nivel
    campaigns = list(df['campaña'].unique())
    channels = list(df['canal'].unique())
    regions = list(df['region'].unique())

    # Crear mapeo de nombres a índices
    all_nodes = campaigns + channels + regions
    node_dict = {node: i for i, node in enumerate(all_nodes)}

    # Crear conexiones (source, target, value)
    sources = []
    targets = []
    values = []

    # Campaña -> Canal
    campaign_to_channel = df.groupby(['campaña', 'canal'])['ventas_euros'].sum().reset_index()
    for _, row in campaign_to_channel.iterrows():
        sources.append(node_dict[row['campaña']])
        targets.append(node_dict[row['canal']])
        values.append(row['ventas_euros'])

    # Canal -> Región
    channel_to_region = df.groupby(['canal', 'region'])['ventas_euros'].sum().reset_index()
    for _, row in channel_to_region.iterrows():
        sources.append(node_dict[row['canal']])
        targets.append(node_dict[row['region']])
        values.append(row['ventas_euros'])

    return all_nodes, sources, targets, values

# Preparar los datos
nodes, sources, targets, values = prepare_sankey_data(df)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] * 10

# Crear figura Sankey
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = colors[:len(nodes)]
    ),
    link = dict(
        source = sources,
        target = targets,
        value = values,
        color = 'rgba(100, 100, 100, 0.3)'
    )
)])

fig.update_layout(
    title_text="Flujo de Ventas: Campaña → Canal → Región",
    font_size=12,
    height=700,
    width=1000
)

# Crear la app
app_2 = dash.Dash()

app_2.layout = html.Div([
    html.H1("Análisis de Flujo de Ventas",
            style={'text-align': 'center', 'margin-bottom': '30px'}),

    dcc.Graph(
        id='sankey-diagram',
        figure=fig
    ),
], style={'background-color': 'white', 'padding': '20px'})

app_2.run(debug=True,port=8053,jupyter_height=1000, jupyter_width=1500)

<IPython.core.display.Javascript object>

In [9]:
# Calcular ROI aproximado por campaña y canal

def calculate_roi(df):
    roi_data = df.groupby(['campaña', 'canal']).agg({
        'ventas_euros': 'sum',
        'unidades_vendidas': 'sum',
        'precio_unitario': 'mean'
    }).reset_index()


    # Asumimos que el costo es 70% del precio de venta
    roi_data['costo_estimado'] = roi_data['precio_unitario'] * roi_data['unidades_vendidas'] * 0.7
    roi_data['roi'] = ((roi_data['ventas_euros'] - roi_data['costo_estimado']) / roi_data['costo_estimado']) * 100
    roi_data['roi'] = roi_data['roi'].round(1)

    return roi_data


roi_df = calculate_roi(df)

# Crear la app
app_3 = dash.Dash()

app_3.layout = html.Div([
    html.H1("ROI por Campaña y Canal",
            style={'text-align': 'center', 'margin-bottom': '30px'}),

    # Selector de vista
    html.Div([
        html.Label("Tipo de Visualización:", style={'font-weight': 'bold','background-color': 'white'}),
        dcc.RadioItems(
            id='chart-type',
            options=[
                {'label': 'Barras Agrupadas', 'value': 'bar'},
                {'label': 'Scatter Plot', 'value': 'scatter'}
            ],
            value='heatmap',
            inline=True,
            style={'margin': '10px 0','background-color': 'white'}
        )
    ], style={'text-align': 'center', 'margin-bottom': '30px',}),

    # Gráfico principal
    dcc.Graph(id='roi-chart'),

    # Estadísticas resumen
    html.Div(id='roi-stats', style={'margin-top': '30px'}),

],style={'background-color': 'white'})

# Callback para actualizar el gráfico
@app_3.callback(
    [Output('roi-chart', 'figure'),
     Output('roi-stats', 'children')],
    Input('chart-type', 'value')
)
def update_roi_chart(chart_type):

    if chart_type == 'bar':
        # Gráfico de barras agrupadas
        fig = px.bar(roi_df,
                    x='campaña',
                    y='roi',
                    color='canal',
                    title='ROI (%) por Campaña y Canal',
                    labels={'roi': 'ROI (%)', 'campaña': 'Campaña'},
                    text='roi')

        fig.update_traces(texttemplate='%{text}%', textposition='outside')
        fig.update_layout(height=500, width=1000, xaxis_tickangle=-45)

    else:
        # Scatter plot con tamaño por ventas
        fig = px.scatter(roi_df,
                        x='ventas_euros',
                        y='roi',
                        size='unidades_vendidas',
                        color='canal',
                        hover_name='campaña',
                        title='ROI vs Ventas por Canal',
                        labels={'roi': 'ROI (%)', 'ventas_euros': 'Ventas (€)'})

        fig.update_layout(height=500, width=900)

    # Crear estadísticas resumen
    best_campaign = roi_df.loc[roi_df['roi'].idxmax()]
    worst_campaign = roi_df.loc[roi_df['roi'].idxmin()]
    avg_roi_by_channel = roi_df.groupby('canal')['roi'].mean().round(1)

    stats = html.Div([
        html.H3("KPIs", style={'text-align': 'center'}),
        html.Div([
            html.Div([
                html.H4("Mejor Performance:", style={'color': 'green'}),
                html.P(f"{best_campaign['campaña']} - {best_campaign['canal']}"),
                html.P(f"ROI: {best_campaign['roi']}%", style={'font-weight': 'bold'})
            ], style={'display': 'inline-block', 'width': '30%', 'text-align': 'center', 'vertical-align': 'top'}),

            html.Div([
                html.H4("Peor Performance:", style={'color': 'red'}),
                html.P(f"{worst_campaign['campaña']} - {worst_campaign['canal']}"),
                html.P(f"ROI: {worst_campaign['roi']}%", style={'font-weight': 'bold'})
            ], style={'display': 'inline-block', 'width': '30%', 'text-align': 'center', 'vertical-align': 'top'}),

            html.Div([
                html.H4("ROI Promedio por Canal:", style={'color': 'blue'}),
                html.Ul([
                    html.Li(f"{canal}: {roi}%")
                    for canal, roi in avg_roi_by_channel.head(3).items()
                ])
            ], style={'display': 'inline-block', 'width': '40%', 'text-align': 'left', 'vertical-align': 'top'})
        ], style={'display': 'flex', 'justify-content': 'space-around'})
    ])

    return fig, stats

app_3.run(debug=True, port=8054,jupyter_height=1000, jupyter_width=1500)

<IPython.core.display.Javascript object>

In [10]:

# Calcular precio medio por región y canal
precio_data = df.groupby(['region', 'canal'])['precio_unitario'].mean().reset_index()
precio_data['precio_unitario'] = precio_data['precio_unitario'].round(2)

# Crear la app
app_4 = dash.Dash(__name__)

app_4.layout = html.Div([
    html.H1("Análisis de Precio Medio por Región y Canal",
            style={
                'text-align': 'center',
                'margin-bottom': '30px',
                'background-color': 'white',
                'padding': '10px',
                'border-radius': '5px'
            }),

    # Controles
    html.Div([
        html.Div([
            html.Label("Tipo de Visualización:", style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='precio-chart-type',
                options=[
                    {'label': 'Heatmap', 'value': 'heatmap'},
                    {'label': 'Barras Agrupadas', 'value': 'bar'},
                    {'label': 'Box Plot', 'value': 'box'}
                ],
                value='heatmap',
                inline=True,
                style={'margin': '10px 0'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Filtrar por Canal:", style={'font-weight': 'bold'}),
            dcc.Dropdown(
                id='canal-filter',
                options=[{'label': 'Todos los Canales', 'value': 'Todos'}] +
                        [{'label': canal, 'value': canal} for canal in sorted(df['canal'].unique())],
                value='Todos',
                style={'margin': '10px 0'}
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ], style={'margin-bottom': '30px'}),

    # Gráfico principal
    dcc.Graph(id='precio-chart'),

    # Estadísticas resumen
    html.Div(id='precio-stats', style={'margin-top': '30px'})

], style={'background-color': 'white', 'padding': '20px'})

# Callback para actualizar el gráfico
@app_4.callback(
    [Output('precio-chart', 'figure'),
     Output('precio-stats', 'children')],
    [Input('precio-chart-type', 'value'),
     Input('canal-filter', 'value')]
)
def update_precio_chart(chart_type, canal_filter):

    # Filtrar datos según canal seleccionado
    if canal_filter == 'Todos':
        filtered_df = df
        filtered_precio_data = precio_data
        title_suffix = ""
    else:
        filtered_df = df[df['canal'] == canal_filter]
        filtered_precio_data = precio_data[precio_data['canal'] == canal_filter]
        title_suffix = f" - {canal_filter}"

    if chart_type == 'heatmap':
        # Heatmap de precios medios
        pivot_precio = precio_data.pivot(index='region', columns='canal', values='precio_unitario')
        pivot_precio = pivot_precio.fillna(0)

        fig = px.imshow(pivot_precio,
                       title=f'Precio Medio (€) por Región y Canal{title_suffix}',
                       labels=dict(x="Canal", y="Región", color="Precio (€)"),
                       color_continuous_scale='Viridis',
                       aspect="auto")

        fig.update_layout(height=500, width=1000)

    elif chart_type == 'bar':
        # Barras agrupadas
        if canal_filter == 'Todos':
            fig = px.bar(precio_data,
                        x='region',
                        y='precio_unitario',
                        color='canal',
                        title=f'Precio Medio (€) por Región y Canal{title_suffix}',
                        labels={'precio_unitario': 'Precio Medio (€)', 'region': 'Región'},
                        text='precio_unitario')
        else:
            fig = px.bar(filtered_precio_data,
                        x='region',
                        y='precio_unitario',
                        title=f'Precio Medio (€) por Región{title_suffix}',
                        labels={'precio_unitario': 'Precio Medio (€)', 'region': 'Región'},
                        text='precio_unitario')

        fig.update_traces(texttemplate='€%{text}', textposition='outside')
        fig.update_layout(height=500, width=1000, xaxis_tickangle=-45)

    elif chart_type == 'box':
        # Box plot por región
        fig = px.box(filtered_df,
                    x='region',
                    y='precio_unitario',
                    color='canal' if canal_filter == 'Todos' else None,
                    title=f'Distribución de Precios por Región{title_suffix}',
                    labels={'precio_unitario': 'Precio Unitario (€)', 'region': 'Región'})

        fig.update_layout(height=500, width=1000, xaxis_tickangle=-45)

    # Crear estadísticas resumen
    if canal_filter == 'Todos':
        stats_df = precio_data
    else:
        stats_df = filtered_precio_data

    region_mas_cara = stats_df.loc[stats_df['precio_unitario'].idxmax()]
    region_mas_barata = stats_df.loc[stats_df['precio_unitario'].idxmin()]
    precio_promedio_general = stats_df['precio_unitario'].mean()

    # Precios por canal
    precio_por_canal = df.groupby('canal')['precio_unitario'].mean().round(2).sort_values(ascending=False)

    stats = html.Div([
        html.H3("KPIs", style={'text-align': 'center'}),
        html.Div([
            html.Div([
                html.H4("Región Más Cara:", style={'color': 'red'}),
                html.P(f"{region_mas_cara['region']}"),
                html.P(f"€{region_mas_cara['precio_unitario']}", style={'font-weight': 'bold'})
            ], style={'display': 'inline-block', 'width': '25%', 'text-align': 'center', 'vertical-align': 'top'}),

            html.Div([
                html.H4("Región Más Barata:", style={'color': 'green'}),
                html.P(f"{region_mas_barata['region']}"),
                html.P(f"€{region_mas_barata['precio_unitario']}", style={'font-weight': 'bold'})
            ], style={'display': 'inline-block', 'width': '25%', 'text-align': 'center', 'vertical-align': 'top'}),

            html.Div([
                html.H4("Precio Promedio:", style={'color': 'blue'}),
                html.P(f"€{precio_promedio_general:.2f}", style={'font-size': '20px', 'font-weight': 'bold'})
            ], style={'display': 'inline-block', 'width': '25%', 'text-align': 'center', 'vertical-align': 'top'}),

            html.Div([
                html.H4("Top 3 Canales (Precio):", style={'color': 'purple'}),
                html.Ul([
                    html.Li(f"{canal}: €{precio}")
                    for canal, precio in precio_por_canal.head(3).items()
                ])
            ], style={'display': 'inline-block', 'width': '25%', 'text-align': 'left', 'vertical-align': 'top'})
        ], style={'display': 'flex', 'justify-content': 'space-around'})
    ], style={'background-color': '#f8f9fa', 'padding': '20px', 'border-radius': '10px'})

    return fig, stats

app_4.run(debug=True, port=8055,jupyter_height=1000, jupyter_width=1500)

<IPython.core.display.Javascript object>